In [1]:
import random
import pandas as pd
import numpy as np
import Covid19_model as CM
import matplotlib.pyplot as plt
import numpy as np
import shutil
import sys
import os.path
import math
import csv
from pyomo.environ import *
import lhsmdu
import time

# Set parameter

In [2]:
D = 1e2
n_site = 16
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 2.7e6

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
action_store = []
random.seed(1)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne, num_sample = 0, 0, 0, 20000
done = False
index = [0]
start_time = 6
for t in range(T):
    pset = env.para_sampling(100)
    para = pset[index[0]]
    action_buffer = [] 
    reward_buffer = []
    
    if budget > 0 and t> start_time:     
        for j in range(num_sample):  
            b0 = budget
            s0 = score
            ac = [random.random(),random.random(),random.random(),random.random(),random.random()]
            ac = [ac * n_site]
            ac = np.reshape(np.array(ac), (n_site, 5)).T
            
            action_buffer.append(ac) 
            state_ex, r, _, _, c = env.transition(state, ac, para)  
            s0 += r
            b0 -= c
            for i in range(t, T):
                if b0 > 0:
                    act = np.ones((5, n_site)) 
                    
                else:
                    act = np.zeros((5, n_site))
                next_ex, reward_ex, _, _, c = env.transition(state_ex, act, para)  
                s0 += reward_ex
                b0 -= c
                state_ex = next_ex.copy()          
            reward_buffer.append(s0)
               
        max_reward = np.argmax(np.array(reward_buffer))
        action = action_buffer[max_reward]
    else:
        action = np.zeros((5, n_site))
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, pset, 100, 1)

    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e
    action_store.append(action)
    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]), 
                  budget, score))
    print('\r', 'Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, reward:{:.2f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]), 
                  np.sum(state[4]), np.sum(state[5]), budget, score))
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    print(action)
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-229.27
 Time: 0 | S: 25495888.0, E: 112.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-229.27
 Time: 0 | new_e:52.9, cum_e:52.9, new_i:47.3, cum_i:47.3
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
 Time: 1 | S: 25495799.8, E: 147.8, A: 0.0, I: 47.3, D: 0.0, R: 5.1, Budget: 2700000.00, reward:-554.35
 Time: 1 | S: 25495797.0, E: 143.0, A: 0.0, I: 47.3, D: 0.0, R: 0.0, Budget: 2700000.00, reward:-554.35
 Time: 1 | new_e:74.3, cum_e:127.2, new_i:65.4, cum_i:112.6
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 

 Time: 11 | S: 25372360.0, E: 551.6, A: 1037.4, I: 2093.8, D: 4.7, R: 119952.5, Budget: 2548233.13, reward:-13176.53
 Time: 11 | S: 25372484.0, E: 526.0, A: 1037.4, I: 2093.8, D: 4.7, R: 119940.0, Budget: 2548233.13, reward:-13176.53
 Time: 11 | new_e:270.8, cum_e:2960.8, new_i:350.8, cum_i:2995.8
[[0.2788997  0.2788997  0.2788997  0.2788997  0.2788997  0.2788997
  0.2788997  0.2788997  0.2788997  0.2788997  0.2788997  0.2788997
  0.2788997  0.2788997  0.2788997  0.2788997 ]
 [0.59610213 0.59610213 0.59610213 0.59610213 0.59610213 0.59610213
  0.59610213 0.59610213 0.59610213 0.59610213 0.59610213 0.59610213
  0.59610213 0.59610213 0.59610213 0.59610213]
 [0.97678602 0.97678602 0.97678602 0.97678602 0.97678602 0.97678602
  0.97678602 0.97678602 0.97678602 0.97678602 0.97678602 0.97678602
  0.97678602 0.97678602 0.97678602 0.97678602]
 [0.99795483 0.99795483 0.99795483 0.99795483 0.99795483 0.99795483
  0.99795483 0.99795483 0.99795483 0.99795483 0.99795483 0.99795483
  0.99795483 0.997

 Time: 18 | S: 25189586.5, E: 195.3, A: 1047.1, I: 2544.5, D: 11.8, R: 302614.8, Budget: 2351400.72, reward:-18353.17
 Time: 18 | S: 25189759.0, E: 188.0, A: 1047.1, I: 2544.5, D: 11.8, R: 302612.0, Budget: 2351400.72, reward:-18353.17
 Time: 18 | new_e:114.3, cum_e:4065.2, new_i:209.3, cum_i:4823.6
[[0.90060964 0.90060964 0.90060964 0.90060964 0.90060964 0.90060964
  0.90060964 0.90060964 0.90060964 0.90060964 0.90060964 0.90060964
  0.90060964 0.90060964 0.90060964 0.90060964]
 [0.94311497 0.94311497 0.94311497 0.94311497 0.94311497 0.94311497
  0.94311497 0.94311497 0.94311497 0.94311497 0.94311497 0.94311497
  0.94311497 0.94311497 0.94311497 0.94311497]
 [0.95868555 0.95868555 0.95868555 0.95868555 0.95868555 0.95868555
  0.95868555 0.95868555 0.95868555 0.95868555 0.95868555 0.95868555
  0.95868555 0.95868555 0.95868555 0.95868555]
 [0.98600952 0.98600952 0.98600952 0.98600952 0.98600952 0.98600952
  0.98600952 0.98600952 0.98600952 0.98600952 0.98600952 0.98600952
  0.98600952 0

 Time: 25 | S: 25029513.2, E: 431.8, A: 595.8, I: 2253.9, D: 17.0, R: 463188.2, Budget: 2225600.63, reward:-23126.04
 Time: 25 | S: 25029687.0, E: 435.0, A: 595.8, I: 2253.9, D: 17.0, R: 463182.0, Budget: 2225600.63, reward:-23126.04
 Time: 25 | new_e:217.6, cum_e:5091.2, new_i:255.2, cum_i:6283.3
[[0.94786249 0.94786249 0.94786249 0.94786249 0.94786249 0.94786249
  0.94786249 0.94786249 0.94786249 0.94786249 0.94786249 0.94786249
  0.94786249 0.94786249 0.94786249 0.94786249]
 [0.80647933 0.80647933 0.80647933 0.80647933 0.80647933 0.80647933
  0.80647933 0.80647933 0.80647933 0.80647933 0.80647933 0.80647933
  0.80647933 0.80647933 0.80647933 0.80647933]
 [0.94529084 0.94529084 0.94529084 0.94529084 0.94529084 0.94529084
  0.94529084 0.94529084 0.94529084 0.94529084 0.94529084 0.94529084
  0.94529084 0.94529084 0.94529084 0.94529084]
 [0.98578841 0.98578841 0.98578841 0.98578841 0.98578841 0.98578841
  0.98578841 0.98578841 0.98578841 0.98578841 0.98578841 0.98578841
  0.98578841 0.9

 Time: 32 | S: 24915478.7, E: 530.5, A: 519.9, I: 2339.1, D: 23.3, R: 577108.5, Budget: 2111891.06, reward:-28703.39
 Time: 32 | S: 24915653.0, E: 545.0, A: 519.9, I: 2339.1, D: 23.3, R: 577115.0, Budget: 2111891.06, reward:-28703.39
 Time: 32 | new_e:253.9, cum_e:6283.3, new_i:281.8, cum_i:7876.0
[[0.68538276 0.68538276 0.68538276 0.68538276 0.68538276 0.68538276
  0.68538276 0.68538276 0.68538276 0.68538276 0.68538276 0.68538276
  0.68538276 0.68538276 0.68538276 0.68538276]
 [0.56807977 0.56807977 0.56807977 0.56807977 0.56807977 0.56807977
  0.56807977 0.56807977 0.56807977 0.56807977 0.56807977 0.56807977
  0.56807977 0.56807977 0.56807977 0.56807977]
 [0.96946024 0.96946024 0.96946024 0.96946024 0.96946024 0.96946024
  0.96946024 0.96946024 0.96946024 0.96946024 0.96946024 0.96946024
  0.96946024 0.96946024 0.96946024 0.96946024]
 [0.98944482 0.98944482 0.98944482 0.98944482 0.98944482 0.98944482
  0.98944482 0.98944482 0.98944482 0.98944482 0.98944482 0.98944482
  0.98944482 0.9

...

 Time: 60 | S: 24338505.6, E: 66.0, A: 257.4, I: 1230.6, D: 45.8, R: 1155894.7, Budget: 1458952.85, reward:-41399.00
 Time: 60 | S: 24338474.0, E: 74.0, A: 257.4, I: 1230.6, D: 45.8, R: 1155947.0, Budget: 1458952.85, reward:-41399.00
 Time: 60 | new_e:34.7, cum_e:8913.9, new_i:57.7, cum_i:12063.3
[[0.34238881 0.34238881 0.34238881 0.34238881 0.34238881 0.34238881
  0.34238881 0.34238881 0.34238881 0.34238881 0.34238881 0.34238881
  0.34238881 0.34238881 0.34238881 0.34238881]
 [0.75342981 0.75342981 0.75342981 0.75342981 0.75342981 0.75342981
  0.75342981 0.75342981 0.75342981 0.75342981 0.75342981 0.75342981
  0.75342981 0.75342981 0.75342981 0.75342981]
 [0.99980606 0.99980606 0.99980606 0.99980606 0.99980606 0.99980606
  0.99980606 0.99980606 0.99980606 0.99980606 0.99980606 0.99980606
  0.99980606 0.99980606 0.99980606 0.99980606]
 [0.95559146 0.95559146 0.95559146 0.95559146 0.95559146 0.95559146
  0.95559146 0.95559146 0.95559146 0.95559146 0.95559146 0.95559146
  0.95559146 0.95

 Time: 67 | S: 24222100.7, E: 7.5, A: 156.3, I: 722.7, D: 47.4, R: 1272965.4, Budget: 1261487.95, reward:-41846.72
 Time: 67 | S: 24222049.0, E: 12.0, A: 156.3, I: 722.7, D: 47.4, R: 1273015.0, Budget: 1261487.95, reward:-41846.72
 Time: 67 | new_e:4.3, cum_e:8996.9, new_i:20.4, cum_i:12293.2
[[0.45012155 0.45012155 0.45012155 0.45012155 0.45012155 0.45012155
  0.45012155 0.45012155 0.45012155 0.45012155 0.45012155 0.45012155
  0.45012155 0.45012155 0.45012155 0.45012155]
 [0.80730602 0.80730602 0.80730602 0.80730602 0.80730602 0.80730602
  0.80730602 0.80730602 0.80730602 0.80730602 0.80730602 0.80730602
  0.80730602 0.80730602 0.80730602 0.80730602]
 [0.99756856 0.99756856 0.99756856 0.99756856 0.99756856 0.99756856
  0.99756856 0.99756856 0.99756856 0.99756856 0.99756856 0.99756856
  0.99756856 0.99756856 0.99756856 0.99756856]
 [0.98815891 0.98815891 0.98815891 0.98815891 0.98815891 0.98815891
  0.98815891 0.98815891 0.98815891 0.98815891 0.98815891 0.98815891
  0.98815891 0.988158

 Time: 74 | S: 24066959.5, E: 3.3, A: 62.3, I: 383.3, D: 48.4, R: 1428543.2, Budget: 1025838.64, reward:-41990.80
 Time: 74 | S: 24066912.0, E: 0.0, A: 62.3, I: 383.3, D: 48.4, R: 1428587.0, Budget: 1025838.64, reward:-41990.80
 Time: 74 | new_e:2.2, cum_e:9019.4, new_i:8.5, cum_i:12384.0
[[0.85167474 0.85167474 0.85167474 0.85167474 0.85167474 0.85167474
  0.85167474 0.85167474 0.85167474 0.85167474 0.85167474 0.85167474
  0.85167474 0.85167474 0.85167474 0.85167474]
 [0.88450176 0.88450176 0.88450176 0.88450176 0.88450176 0.88450176
  0.88450176 0.88450176 0.88450176 0.88450176 0.88450176 0.88450176
  0.88450176 0.88450176 0.88450176 0.88450176]
 [0.98018757 0.98018757 0.98018757 0.98018757 0.98018757 0.98018757
  0.98018757 0.98018757 0.98018757 0.98018757 0.98018757 0.98018757
  0.98018757 0.98018757 0.98018757 0.98018757]
 [0.99500378 0.99500378 0.99500378 0.99500378 0.99500378 0.99500378
  0.99500378 0.99500378 0.99500378 0.99500378 0.99500378 0.99500378
  0.99500378 0.99500378 0

 Time: 81 | S: 23773247.1, E: 1.3, A: 24.1, I: 190.1, D: 48.8, R: 1722488.8, Budget: 781845.46, reward:-42042.79
 Time: 81 | S: 23773209.0, E: 0.0, A: 24.1, I: 190.1, D: 48.8, R: 1722530.0, Budget: 781845.46, reward:-42042.79
 Time: 81 | new_e:0.6, cum_e:9026.4, new_i:3.1, cum_i:12418.6
[[0.20783556 0.20783556 0.20783556 0.20783556 0.20783556 0.20783556
  0.20783556 0.20783556 0.20783556 0.20783556 0.20783556 0.20783556
  0.20783556 0.20783556 0.20783556 0.20783556]
 [0.88494578 0.88494578 0.88494578 0.88494578 0.88494578 0.88494578
  0.88494578 0.88494578 0.88494578 0.88494578 0.88494578 0.88494578
  0.88494578 0.88494578 0.88494578 0.88494578]
 [0.99547294 0.99547294 0.99547294 0.99547294 0.99547294 0.99547294
  0.99547294 0.99547294 0.99547294 0.99547294 0.99547294 0.99547294
  0.99547294 0.99547294 0.99547294 0.99547294]
 [0.99773228 0.99773228 0.99773228 0.99773228 0.99773228 0.99773228
  0.99773228 0.99773228 0.99773228 0.99773228 0.99773228 0.99773228
  0.99773228 0.99773228 0.9

 Time: 88 | S: 23596637.2, E: 2.8, A: 9.3, I: 89.6, D: 48.9, R: 1899212.1, Budget: 633165.77, reward:-42081.27
 Time: 88 | S: 23596606.0, E: 0.0, A: 9.3, I: 89.6, D: 48.9, R: 1899245.0, Budget: 633165.77, reward:-42081.27
 Time: 88 | new_e:1.3, cum_e:9033.4, new_i:2.2, cum_i:12435.6
[[0.53379645 0.53379645 0.53379645 0.53379645 0.53379645 0.53379645
  0.53379645 0.53379645 0.53379645 0.53379645 0.53379645 0.53379645
  0.53379645 0.53379645 0.53379645 0.53379645]
 [0.92538568 0.92538568 0.92538568 0.92538568 0.92538568 0.92538568
  0.92538568 0.92538568 0.92538568 0.92538568 0.92538568 0.92538568
  0.92538568 0.92538568 0.92538568 0.92538568]
 [0.99336831 0.99336831 0.99336831 0.99336831 0.99336831 0.99336831
  0.99336831 0.99336831 0.99336831 0.99336831 0.99336831 0.99336831
  0.99336831 0.99336831 0.99336831 0.99336831]
 [0.99336566 0.99336566 0.99336566 0.99336566 0.99336566 0.99336566
  0.99336566 0.99336566 0.99336566 0.99336566 0.99336566 0.99336566
  0.99336566 0.99336566 0.99336

In [6]:
result_state = pd.DataFrame(state_buffer)
result_state.to_csv(r'rollout_high_state_16.csv')
result_new = pd.DataFrame(new_buffer)
result_new.to_csv(r'rollout_high_new_16.csv')

In [11]:
import pickle
pickle.dump(action_store, open("rollout_high_action.pth", 'wb'))

In [8]:
(time.time() - tic) / ( T - start_time + 1)

102.43530717737535

In [9]:
9034.9 + 12437.8

21472.699999999997